In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import re
import pymysql

#創建table
def create_mysql_table(board):
    db = pymysql.connect("mysql","root","123456","pttDB",charset='utf8mb4',)
    cursor = db.cursor()
    sql ="""create table if not exists {}(articleID varchar(16) primary key, 
            authorId varchar(14) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            authorName varchar(14) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci ,
            category varchar(4) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            title varchar(20) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            publishedTime varchar(20) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            content text CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            canonicalUrl varchar(64) CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci,
            createdTime varchar(30),
            updateTime varchar(30),
            commentId longtext,
            commentContent longtext,
            commentTime longtext)
        """.format(board)
    
    cursor.execute(sql)
    db.commit()
    db.close()
    
#插入資料
def insert_data(board,articleID , authorId ,authorName , category ,title ,publishedTime ,content ,canonicalUrl ,createdTime ,updateTime ,commentId ,commentContent ,commentTime,g,page_count):
    db = pymysql.connect("mysql","root","123456","pttDB",charset='utf8mb4',use_unicode = True )
    cursor = db.cursor()
    sql = """insert into {}(articleID,authorId,authorName,category,title,publishedTime,content,
             canonicalUrl,createdTime,updateTime,commentId,commentContent,commentTime) 
             values('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}')""".format(
             board, str(articleID) ,str(authorId) ,str(authorName) , str(category) ,str(title) ,str(publishedTime),str(content) ,str(canonicalUrl) ,str(createdTime) ,
             str(updateTime) ,pymysql.escape_string(str(commentId)) ,pymysql.escape_string(str(commentContent)),pymysql.escape_string(str(commentTime)))
    
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()
        db.close()
    print('存好第',page_count,'頁  第',g,'篇資料')


#時間格式修改(資料建立時間與資料更新時間)
def time_convert(createdTime):
    import time 
    try:
        tempTime = time.strptime(createdTime,'%a %b %d %H:%M:%S %Y')
        resTime = time.strftime('%Y-%m-%dT%H:%M:%S.020+00:00',tempTime)
    except ValueError :
        tempTime = time.strptime(createdTime,' %m/%d/%Y %H:%M:%S')
        resTime = time.strftime('%Y-%m-%dT%H:%M:%S.020+00:00',tempTime)
    return resTime

#時間格式修改(貼文和推文時間改時間戳)
def time_mktime(publishedTime):
    import time
    try:
        resTime = str(round(time.mktime(time.strptime(publishedTime,'%a %b %d %H:%M:%S %Y')),))+"000"
    except Exception:
        resTime = str(round(time.mktime(time.strptime(publishedTime,' %Y %m/%d %H:%M')),))
    return resTime

#比較文章時間的格式修改
def compare_mktime(publishedTime):
    import time
    try:
        resTime = str(round(time.mktime(time.strptime(publishedTime,'%a %b %d %H:%M:%S %Y')),))
    except Exception:
        resTime = str(round(time.mktime(time.strptime(publishedTime,' %Y %m/%d %H:%M')),))
    return resTime

#推文的時間格式修改
def commentTime_mktime(commentTime_year):
    import time
    try:
        resTime = str(round(time.mktime(time.strptime(commentTime_year,' %Y %m/%d %H:%M:%S')),))
    except Exception:
        resTime = str(round(time.mktime(time.strptime(commentTime_year,' %Y %m/%d')),))
    return resTime

#頁數日期修改
def last_mktime(start_date_tmp):
    import time
    resTime = str(round(time.mktime(time.strptime(start_date_tmp,'%Y-%m-%d')),))
    return resTime

#文章內文
def content(artical_tmp):       
    artical_content = artical_tmp[0].text.split('--')[0].replace('\n','').replace(' ','').replace('\t','')  
    print(artical_content)
    return artical_content

#完整推文    
def push_content(content_count):
    comment_word = []
    for count in range(0,len(content_count)):
        commentContent = content_count[count].text.replace(":","")
        comment_word.append(commentContent)
    print(comment_word)
    return comment_word

#推文作者    
def push_commentID(commentID_count):
    comment_user = []
    for count in range(0,len(commentID_count)):
        commentID = commentID_count[count].text
        comment_user.append(commentID)
    print(comment_user)
    return comment_user

#推文時間
def push_commentTime(artical_count,time_year): 
    push_time = []
    for count in range(0,len(artical_count)):
        if len(artical_count[count].text) <= 28 :       #判斷推文是否有時分秒，若有正常抓
            commentTime_tmp = artical_count[count].text.replace('\n','')[-11:]  #推文時間
            commentTime_year = '{} '.format(time_year)+commentTime_tmp  
            commentTime = time_mktime(commentTime_year)
            
        else:
            import time
            result = time.ctime() #若格式不一樣沒有時分秒  就調整格式
            hours = result[-13:-5]
            commentTime_tmp = artical_count[count].text.replace('\n','')[-5:]  #推文時間
            commentTime_year = '{} '.format(time_year)+commentTime_tmp+" "+hours  
            commentTime = commentTime_mktime(commentTime_year)
        push_time.append(commentTime)
    print(push_time)
    return push_time

#擷取總頁數
def allpage_number(board):                                           
    urlpage = 'https://www.ptt.cc/bbs/{}/index.html'.format(board)
    res = requests.get(urlpage,headers = headers, cookies = {'over18':'1'})
    soup = BeautifulSoup(res.text, 'html.parser')
    allpageurl = soup.select('.btn.wide')[1]['href']
    start_index = allpageurl.find('index')
    end_index = allpageurl.find('.html')
    page_number = eval("%s +1" %(allpageurl[start_index + 5 : end_index]))
    print("總頁數",page_number)
    return page_number


"================================================================================================"
'''
[爬下所設定的所有分類看板]
取出設定的其中一個看板，接著取出看板的所有頁數，再將每一頁的每篇文章的標題與網址取出
接著取得文章分類，文章分類有少部分格式不一樣，所以列為其他。
再來爬取貼文時間 文章內容 推文內容 推文作者 推文時間等等 

一開始會針對分類看板建立一個table，抓到下個看板會再建立另外一個table
每抓一篇資料會儲存一次。

'''

#設定抓所有看板的文章
def all_board(class_board):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36'
    headers = {'User-Agent':user_agent}
    for board in class_board:
        create_mysql_table(board)                       #建立mysql的分類table
        page_number = allpage_number(board)
        for page_count in range(page_number,0,-1):      #抓取分類看板的所有頁數，再由最新抓回來
            time_year = 0
            url_allpage = 'https://www.ptt.cc/bbs/{}/index{}.html'.format(board,page_count)
            res = requests.get(url_allpage,headers = headers, cookies = {'over18':'1'})
            soup = BeautifulSoup(res.text, 'html.parser')
            title_url = soup.select('div[class="title"] a')
            print('第',page_count,'頁')
            for g,i in enumerate(title_url) :            #取出分頁中各篇文章的網址
                try:
                    sort_tmp = i.text
                    sort = re.findall(r'\[\w\w',sort_tmp)[0].replace('[','')
                    print('文章分類為:'+ sort)             #文章分類
                except Exception :
                    sort = '其他'
                    print('文章分類為: 其他')
                try:
                    title = i.text.replace(' ','').split(']')[1]     #文章標題
                    print(title)
                except Exception :
                    title = i.text
                    print(title)
                    
                article_id_tmp = i['href']      #文章編號
                end_index = article_id_tmp.find('.html')
                article_id = article_id_tmp[-23:end_index].replace(".","")
                print(article_id)
                
                
                canonicalUrl = 'https://www.ptt.cc'+i['href']    #標準網址
                print(canonicalUrl)
                authorID_tmp = soup.select('div[class="author"] ')            
                authorID = authorID_tmp[g].text  #作者編號
                print('作者標號: '+authorID)
                url1= canonicalUrl
                res1 = requests.get(url1,headers = headers, cookies = {'over18':'1'})
                soup1 = BeautifulSoup(res1.text, 'html.parser')
                author = soup1.select('span[class="article-meta-value"]')
                try:
                    authorName_tmp = author[0].text.replace(')','').split('(')
                    authorName = authorName_tmp[1]
                    print('作者暱稱 '+authorName)      #作者暱稱
                except Exception :
                    authorName = '沒有暱稱'
                    print(authorName)
                try:
                    time = soup1.select('span[class="article-meta-value"]')
                    someTime_tmp = time[3].text
                    time_year = someTime_tmp[-5:]
                    print("年分: "+time_year)
                    publishedTime = time_mktime(someTime_tmp)                   #貼文時間

                    print("貼文時間: "+publishedTime)
                except Exception :
                    publishedTime = '版主公告 沒有時間'
                    print(publishedTime)

                artical_tmp = soup1.select('div[id="main-container"]')
                artical_content = content(artical_tmp)         #文章內文
                content_count = soup1.select('div [class="push"] [class="f3 push-content"]')
                comment_word = push_content(content_count)     #推文內容
                commentID_count = soup1.select('div [class="push"] [class="f3 hl push-userid"]')
                comment_user = push_commentID(commentID_count) #推文作者
                artical_count = soup1.select('div [class="push"] [class="push-ipdatetime"]')
                try:
                    if time_year != 0:
                        push_time = push_commentTime(artical_count,time_year)  #推文時間
                    else:
                        updateTime_tmp11 = soup1.select('span[class="f2"]')
                        updateTime = updateTime_tmp11[3]
                        updateTime_convert = updateTime.text.replace('\n','').split(',')[1]
                        time_year = updateTime_convert[-13:-8]

                        push_time = push_commentTime(artical_count,time_year)   #推文時間
                except ValueError:
                    push_time = "沒有推文時間"
                    print("沒有推文時間")
                
                
                import time
                createdTime = time_convert(time.ctime())
                updateTime = time_convert(time.ctime())
                
              
                
                insert_data(board,article_id , authorID ,authorName , sort ,   #將資料存入mysql
                            title ,publishedTime ,artical_content ,canonicalUrl ,createdTime ,
                            updateTime ,comment_user ,comment_word ,push_time,g,page_count)

                
"================================================================================================"

'''
[指定欲擷取的貼文發布日期區間]
從最舊的文章開始搜尋，每一頁都抓當頁最新那篇文章的時間來與我要抓的起始時間比對，
如果當頁最新的那篇文章大於我要抓的起始時間，就會開始讀取當頁第一篇文章，再去
比較是不是介於我的起始時間與結束時間之中，如果符合再開始抓，避免在同一頁裡有
不符合時間區間的文章。

'''

 #抓設定範圍內的文章
def range_search(start_date,end_date,class_board):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36'
    headers = {'User-Agent':user_agent}
    for board in class_board:
        page_number = allpage_number(board)
        for page_count in range(1,page_number):    #判斷每一頁的最後一篇文章
            url= 'https://www.ptt.cc/bbs/NBA/index{}.html'.format(page_count+1)
            print("第",page_count,"頁")
            res = requests.get(url,headers = headers, cookies = {'over18':'1'})    #請求分類看板首頁
            soup = BeautifulSoup(res.text, 'html.parser')
            last = soup.select('div [class="r-ent"] ')
            tmp_last = 'https://www.ptt.cc'+last[len(last)-1].a["href"]    #取出當前頁最後一篇文章網址
            print(tmp_last)
            try:
                res2 = requests.get(tmp_last,headers = headers, cookies = {'over18':'1'}) 
                soup_list = BeautifulSoup(res2.text, 'html.parser')
                time2 = soup_list.select('span[class="article-meta-value"]')
                someTime_tmp2 = time2[3].text
                lastTime2 = compare_mktime(someTime_tmp2)  #當前頁面 最後一篇文章的貼文時間
                print("最後一篇貼文時間: "+lastTime2)
            except Exception :
                print('貼文時間關閉')

            if start_date <= lastTime2 :    #判斷每一頁的最後一篇文章的時間
                for compare_count in range(0,len(last)):
                    compare_url = 'https://www.ptt.cc'+last[compare_count].a['href']   #取出整頁文章網址
                    print('第',compare_count,'篇文章的URL :'+compare_url)
                    res3 = requests.get(compare_url,headers = headers, cookies = {'over18':'1'})
                    soup_content = BeautifulSoup(res3.text, 'html.parser')
                    try:
                        time3 = soup_content.select('span[class="article-meta-value"]')
                        someTime_tmp3 = time3[3].text
                        lastTime3 = compare_mktime(someTime_tmp3)   #抓出第一篇文章時間
                        print('第一篇文章的時間'+lastTime3)
                    except Exception:
                        continue           #若文章被刪除，則換下一篇
                    if  lastTime3 >= start_date and lastTime3 <= end_date:      #第一篇文章時間大於起始時間就抓整頁文章
                        
                        time_year = 0
                        #for g,i in enumerate(title_url) :
                        try:
                            sort_tmp = last[compare_count].text
                            sort = re.findall(r'\[\w\w',sort_tmp)[0].replace('[','')
                            print('文章分類為:'+ sort)                                     #文章分類
                        except Exception :
                            print('文章分類為: 其他')
                            
                        
                        lastt = soup.select('div[class="title"] a')
                        try:
                            title = lastt[compare_count].text.replace(' ','').split(']')[1]   #文章標題
                            print(title)
                        except Exception :
                            title = lastt[compare_count].text
                            print(title)
                        article_id_tmp = lastt[compare_count]['href']                         #文章編號
                        end_index = article_id_tmp.find('.html')
                        article_id = article_id_tmp[-23:end_index].replace(".","")
                        print(article_id)
                        canonicalUrl = 'https://www.ptt.cc'+lastt[compare_count]['href']      #標準網址
                        print(canonicalUrl)
                        
                        authorID_tmp = soup.select('div[class="author"] ')            
                        authorID = authorID_tmp[compare_count].text                           #作者編號
                        print('作者標號 : '+authorID)
                        author = soup_content.select('span[class="article-meta-value"]')
                        try:
                            authorName_tmp = author[0].text.replace(')','').split('(')
                            authorName = authorName_tmp[1]
                            print('作者暱稱 '+authorName)                                  #作者暱稱
                        except Exception :
                            authorName = '沒有暱稱'
                            print(authorName)
                        try:
                            time = soup_list.select('span[class="article-meta-value"]')
                            someTime_tmp = time[3].text
                            time_year = someTime_tmp[-5:]
                            print("年分: "+time_year)
                            publishedTime = time_mktime(someTime_tmp)                     #貼文時間

                            print("貼文時間: "+publishedTime)
                        except Exception :
                            publishedTime = '版主公告 沒有時間'
                            print(publishedTime)
     
                        artical_tmp = soup_content.select('div[id="main-container"]')
                        artical_content = content(artical_tmp)                      #文章內文
                        content_count = soup_content.select('div [class="push"] [class="f3 push-content"]')
                        comment_word = push_content(content_count)                  #推文內容
                        commentID_count = soup_content.select('div [class="push"] [class="f3 hl push-userid"]')
                        comment_user = push_commentID(commentID_count)              #推文作者
                        artical_count = soup_content.select('div [class="push"] [class="push-ipdatetime"]')
                        try:
                            if time_year != 0:
                                push_time = push_commentTime(artical_count,time_year)#推文時間
                                print('time正確')
                            else:
                                updateTime_tmp11 = soup_content.select('span[class="f2"]')
                                try:
                                    updateTime = updateTime_tmp11[3]
                                    updateTime_convert = updateTime.text.replace('\n','').split(',')[1]
                                    time_year = updateTime_convert[-13:-8]                            
                                    push_time = push_commentTime(artical_count,time_year)#推文時間
                                except Exception:
                                    print('沒有推文時間')
                        except ValueError:
                            print("沒有推文時間")
                    elif lastTime3 < end_date:                             #判斷
                        print('沒到起始時間')

                    else :
                        print('資料抓取完畢return')
                        return                                             #抓完時間內的文章後跳出

            else:
                pass
'==============================================================================================='
            
if __name__ =='__main__':
    
    db = pymysql.connect("mysql","root","123456","", )       #與mysql連線並創建資料庫
    cursor = db.cursor()
    cursor.execute("create database if not exists pttDB")
    db = pymysql.connect("mysql","root","123456","pttDB",charset='utf8mb4' )
    cursor.execute("use pttDB")
    db.commit()
    db.close()

    start_date_tmp = input(str('起始時間 格式為2020-01-01 : '))#要抓的起始時間
    end_date_tmp   = input(str('結束時間 格式為2020-01-01 : '))#要抓的結束時間
    #如果兩個都沒輸入就直接全抓
    
    if start_date_tmp == '' and end_date_tmp == '' :
        print('---------------------全部都抓-----------------')
        class_board = ['nCoV2019','NBA','Joke','Gossiping'] #可設定要抓哪幾個版
        df = all_board(class_board)
    else:
        try:
            start_date = last_mktime(start_date_tmp)
            end_date = last_mktime(end_date_tmp)
            print(start_date)
            print(end_date)                     
            class_board = ['NBA']  # 未來可增加抓多個版的功能
            print('--------------------抓區間------------------')
            range_search(start_date,end_date,class_board) 
        except Exception :
            print('輸入不正確')
            
    


In [ ]:
#可印出區間內所有日期
'''import datetime
from datetime import datetime
from dateutil.parser import parse
published_time = datetime.now()
a = '2019-07-08'
startdate = parse("{}".format(a))
enddate = parse("2020-6-8")'''

    
